This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
top500 = pd.read_csv('top500.csv')
top500.head()

,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,166,3,16,325,NaN,NaN,5,101668,1,-1,...,411.0,92,2091962,165908,564317,1484,1.2,9.7,Vardy,1
1,192,4,11,235,100.0,100.0,3,110979,0,0,...,403.0,83,2057103,37727,765571,4405,0.5,6.9,Mané,1
2,460,2,16,277,100.0,100.0,4,173879,0,0,...,501.0,81,3649771,55486,529166,8749,0.9,10.3,Abraham,2
3,215,10,10,317,100.0,100.0,5,61366,0,0,...,291.0,76,2973021,16478,1486213,21234,0.3,7.5,De Bruyne,0
4,191,3,8,204,100.0,100.0,3,118748,0,0,...,640.0,72,932739,12136,1950069,14853,0.3,5.9,Salah,0


In [3]:
histories = {}

ids = top500['id_']

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history   

In [4]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['gameweek'] = [i+1 for i in range((len(df.index)))]
    df['id_'] = key
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    history_dfs[key] = df
    
    
    

In [5]:
type(history_dfs)

dict

In [6]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(5860, 34)

In [7]:
top500histories.to_csv('top500histories.csv')

In [8]:
top500histories.head()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,gameweek,id_,sum_points
0,0,0,5,1,4.9,166,5,0,0,1.6,...,2,0,0,0,90,True,0,1,166,2
1,0,0,-3,0,2.3,166,14,1,0,0.9,...,2,-82257,18334,100591,90,False,0,2,166,4
2,0,2,25,0,1.8,166,27,1,1,6.2,...,8,-78612,32101,110713,89,False,0,3,166,12
3,1,3,63,0,13.6,166,36,1,2,14.6,...,16,-7210,49194,56404,89,True,0,4,166,28
4,0,0,5,0,11.3,166,45,1,0,1.7,...,2,83507,136063,52556,89,False,0,5,166,30
